In [1]:
# CS 6316 ML Project: LASSO
# Created on Wed Apr 29 22:23 2020 
# @author: sl4bq

import time
import os
import shutil
import pandas as pd
from datetime import datetime
from dateutil import tz
import warnings
import itertools

import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [2]:
# Remove the NaN rows and columns

def removeNaN(curr_feat_1, curr_feat_2):
    curr_feat_1_nan = np.array([])  # numpy.float64
    curr_feat_2_nan = np.array([])  # numpy.float64
    posi = 0
    # *** Check for and remove the Nan rows in from ONLY [curr_feat_2] feature rows
    for posi in range(len(curr_feat_2)):
        if not (math.isnan(curr_feat_2[posi])):# and not(math.isnan(curr_feat_1[posi]))):
            curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
        if (math.isnan(curr_feat_1[posi])):
            # curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            # curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
    return curr_feat_1_nan, curr_feat_2_nan

#### Import .csv dataset into pandas
print('Importing Data Now')

#filename = '/Users/Mandala/PycharmProjects/ML_Project/windowing_results.csv'
filename = 'skin_conductance_new.csv'

# Read data from csv file 
df_feat = pd.read_csv(filename)
df_keys = df_feat.keys()



Importing Data Now


In [3]:
df_feat

,Unnamed: 0,Unnamed: 0.1,GSR #,start_time,end_time,gsr_mean,gsr_median,gsr_variance,gsr_skewness,gsr_kurtosis,Act
0,0,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:27,2019-11-21 09:35:29,1.355378,1.332223,0.000256,0.699293,-0.627312,"(0,0)"
1,1,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:28,2019-11-21 09:35:30,1.339162,1.318903,0.000125,-0.100332,-1.238669,"(0,0)"
2,2,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:29,2019-11-21 09:35:31,1.326166,1.306471,0.000135,0.010484,-1.243451,"(0,0)"
3,3,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:30,2019-11-21 09:35:32,1.313587,1.294483,0.000113,0.140383,-1.149240,"(0,0)"
4,4,GSR1_GSR_Skin_Conductance_CAL,1,2019-11-21 09:35:31,2019-11-21 09:35:33,1.302077,1.286491,0.000085,0.139074,-1.145452,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...
10286,10286,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:35,2019-11-21 11:00:37,1.873922,1.857143,0.000018,1.241958,5.575090,"(2,3)"
10287,10287,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:36,2019-11-21 11:00:38,1.872439,1.838095,0.000022,0.428993,5.263945,"(2,3)"
10288,10288,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:37,2019-11-21 11:00:39,1.869794,1.838095,0.000021,0.490440,4.582487,"(2,3)"
10289,10289,GSR6_GSR_Skin_Conductance_CAL,6,2019-11-21 11:00:38,2019-11-21 11:00:40,1.866879,1.838095,0.000020,0.256191,3.974350,"(2,3)"


In [4]:
print(df_keys)
len(df_keys)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'GSR #', 'start_time', 'end_time',
       'gsr_mean', 'gsr_median', 'gsr_variance', 'gsr_skewness',
       'gsr_kurtosis', 'Act'],
      dtype='object')


11

In [6]:
# Data Preprocessing
df = df_feat.copy()
df = df.dropna()

Xs = df[df_keys[5:10]]
y = df['Act']

print(Xs)
print(y)

       gsr_mean  gsr_median  gsr_variance  gsr_skewness  gsr_kurtosis
0      1.355378    1.332223      0.000256      0.699293     -0.627312
1      1.339162    1.318903      0.000125     -0.100332     -1.238669
2      1.326166    1.306471      0.000135      0.010484     -1.243451
3      1.313587    1.294483      0.000113      0.140383     -1.149240
4      1.302077    1.286491      0.000085      0.139074     -1.145452
...         ...         ...           ...           ...           ...
10286  1.873922    1.857143      0.000018      1.241958      5.575090
10287  1.872439    1.838095      0.000022      0.428993      5.263945
10288  1.869794    1.838095      0.000021      0.490440      4.582487
10289  1.866879    1.838095      0.000020      0.256191      3.974350
10290  1.865581    1.842491      0.000017      0.676028      4.517975

[10291 rows x 5 columns]
0        (0,0)
1        (0,0)
2        (0,0)
3        (0,0)
4        (0,0)
         ...  
10286    (2,3)
10287    (2,3)
10288    (2,3)

In [7]:
# This is the data split

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=1)


In [8]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier

m = RandomForestClassifier(n_estimators=20, random_state=0)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[ 33   6   3   2   0   4   2   2   0   0   1   1]
 [  2 154  19  22   1   2   6  11   0   3   3   2]
 [  0  20 164   9   2  13   3   7   0   3   5   1]
 [  0  17  19 109   6   5  11  12   1   3   7   8]
 [  0   2   0   4  11   2   3   1   1   4   5   1]
 [  4   5   8   2   0  90  16  15   0   8  10  11]
 [  0   7   1   4   2  19 198   5   3   8  47  26]
 [  3   9   8   4   2  22  18 106   2  10   9  13]
 [  0   0   1   2   1   2   5   7   6   7  11   1]
 [  1   8   3   3   0  10  11  11   2  56  26  10]
 [  0   7   4   9   0   4  51  15   1  16 152  16]
 [  1   4   1   4   0  10  23   8   0  15  40  61]]
0.5536668285575522


In [11]:
df_video = df.loc[(df.Act == "(0,0)") | (df.Act == "(0,1)") | (df.Act == "(0,2)") | (df.Act == "(0,3)")]
df_noeval = df.loc[(df.Act == "(1,0)") | (df.Act == "(1,1)") | (df.Act == "(1,2)") | (df.Act == "(1,3)")]
df_eval = df.loc[(df.Act == "(2,0)") | (df.Act == "(2,1)") | (df.Act == "(2,2)") | (df.Act == "(2,3)")]

In [12]:
def RFC(x):
    z = x.copy()
    df = z.dropna()

    Xs = df[df_keys[5:10]]
    y = df['Act']
    
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)
    
    m = RandomForestClassifier(n_estimators=20, random_state=0)
    m.fit(X_train, y_train)

    y_pred = m.predict(X_test)

    print(confusion_matrix(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))

In [13]:
RFC(df_video)

[[ 50   9   6   4]
 [  3 159  22  23]
 [  3  26 170  21]
 [  1  28  14 142]]
0.7650513950073421


In [14]:
RFC(df_noeval)

[[ 23   9  10   1]
 [  2 139  30  31]
 [  4  11 260  18]
 [  1  18  39 139]]
0.763265306122449


In [15]:
RFC(df_eval)

[[ 10   9  10   6]
 [ 10 107  36  16]
 [  5  33 209  32]
 [  1  20  40 100]]
0.6614906832298136
